## Types of Plagiarism

Each text file in the data set is associated with one **Task** (task A-E) and one **Category** of plagiarism, which you can see in the above DataFrame.

###  Tasks, A-E

Each text file contains an answer to one short question; these questions are labeled as tasks A-E. For example, Task A asks the question: "What is inheritance in object oriented programming?"

### Categories of plagiarism 

Each text file has an associated plagiarism label/category:

**1. Plagiarized categories: `cut`, `light`, and `heavy`.**
* These categories represent different levels of plagiarized answer texts. `cut` answers copy directly from a source text, `light` answers are based on the source text but include some light rephrasing, and `heavy` answers are based on the source text, but *heavily* rephrased (and will likely be the most challenging kind of plagiarism to detect).
     
**2. Non-plagiarized category: `non`.** 
* `non` indicates that an answer is not plagiarized; the Wikipedia source text is not used to create this answer.
    
**3. Special, source text category: `orig`.**
* This is a specific category for the original, Wikipedia source text. We will use these files only for comparison purposes.

## Read in the Data

This data is a slightly modified version of a dataset created by Paul Clough (Information Studies) and Mark Stevenson (Computer Science), at the University of Sheffield. You can read all about the data collection and corpus, at [their university webpage](https://ir.shef.ac.uk/cloughie/resources/plagiarism_corpus.html). 

> **Citation for data**: Clough, P. and Stevenson, M. Developing A Corpus of Plagiarised Short Answers, Language Resources and Evaluation: Special Issue on Plagiarism and Authorship Analysis, In Press. [Download]

In [ ]:
import os
import methods 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c4147f9_data/data.zip
!unzip data

This plagiarism dataset is made of multiple text files; each of these files has characteristics that are is summarized in a `.csv` file named `file_information.csv`, which we can read in using `pandas`.

In [4]:
csv_file = 'data/file_information.csv'

plagiarism_df = pd.read_csv(csv_file)
plagiarism_df.head()

,File,Task,Category
0,g0pA_taska.txt,a,non
1,g0pA_taskb.txt,b,cut
2,g0pA_taskc.txt,c,light
3,g0pA_taskd.txt,d,heavy
4,g0pA_taske.txt,e,non


In [5]:
def numerical_dataframe(csv_file='data/file_information.csv'):
    
    '''
    Reads in a csv file which is assumed to have `File`, `Category` and `Task` columns.
    This function does two things: 
       1) converts `Category` column values to numerical values 
       2) Adds a new, numerical `Class` label column.
    The `Class` column will label plagiarized answers as 1 and non-plagiarized as 0. 
    Source texts have a special label, -1.
    
    :param csv_file: 
        The directory for the file_information.csv file
    :return: 
        A dataframe with numerical categories and a new `Class` label column
    '''
    
    classifier = {'non': 0, 'heavy': 1, 'light': 1, 'cut': 1, 'orig': -1}
    category = {'non': 0, 'heavy': 1, 'light': 2, 'cut': 3, 'orig': -1}
    
    df = pd.read_csv(csv_file)
    
    df['Class'] = df['Category'].apply(lambda x: classifier.get(x, ''))
    df['Category'] = df['Category'].apply(lambda x: category.get(x, ''))
    
    return df

In [6]:
transformed_df = numerical_dataframe(csv_file ='data/file_information.csv')
transformed_df.head(5)

,File,Task,Category,Class
0,g0pA_taska.txt,a,0,0
1,g0pA_taskb.txt,b,3,1
2,g0pA_taskc.txt,c,2,1
3,g0pA_taskd.txt,d,1,1
4,g0pA_taske.txt,e,0,0


In [8]:
text_df = methods.create_text_column(transformed_df)
text_df.head()

,File,Task,Category,Class,Text
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...


In [9]:
row_idx = 0
sample_text = text_df.iloc[row_idx]['Text']

print('Sample processed text:\n\n', sample_text)

Sample processed text:

 inheritance is a basic concept of object oriented programming where the basic idea is to create new classes that add extra detail to existing classes this is done by allowing the new classes to reuse the methods and variables of the existing classes and new methods and classes are added to specialise the new class inheritance models the is kind of relationship between entities or objects  for example postgraduates and undergraduates are both kinds of student this kind of relationship can be visualised as a tree structure where student would be the more general root node and both postgraduate and undergraduate would be more specialised extensions of the student node or the child nodes  in this relationship student would be known as the superclass or parent class whereas  postgraduate would be known as the subclass or child class because the postgraduate class extends the student class  inheritance can occur on several layers where if visualised would display a l

## Split data into training and test sets

The next cell will add a `Datatype` column to a given DataFrame to indicate if the record is: 
* `train` - Training data, for model training.
* `test` - Testing data, for model evaluation.
* `orig` - The task's original answer from wikipedia.

### Stratified sampling

The given code uses a helper function which you can view in the `methods.py` file in the main project directory. This implements [stratified random sampling](https://en.wikipedia.org/wiki/Stratified_sampling) to randomly split data by task & plagiarism amount. Stratified sampling ensures that we get training and test data that is fairly evenly distributed across task & plagiarism combinations. Approximately 26% of the data is held out for testing and 74% of the data is used for training.

The function **train_test_dataframe** takes in a DataFrame that it assumes has `Task` and `Category` columns, and, returns a modified frame that indicates which `Datatype` (train, test, or orig) a file falls into. This sampling will change slightly based on a passed in *random_seed*. Due to a small sample size, this stratified random sampling will provide more stable results for a binary plagiarism classifier. Stability here is smaller *variance* in the accuracy of classifier, given a random seed.

In [10]:
random_seed = 1

complete_df = methods.train_test_dataframe(text_df, random_seed=random_seed)
complete_df.head(10)

,File,Task,Category,Class,Text,Datatype
0,g0pA_taska.txt,a,0,0,inheritance is a basic concept of object orien...,train
1,g0pA_taskb.txt,b,3,1,pagerank is a link analysis algorithm used by ...,test
2,g0pA_taskc.txt,c,2,1,the vector space model also called term vector...,train
3,g0pA_taskd.txt,d,1,1,bayes theorem was names after rev thomas bayes...,train
4,g0pA_taske.txt,e,0,0,dynamic programming is an algorithm design tec...,train
5,g0pB_taska.txt,a,0,0,inheritance is a basic concept in object orien...,train
6,g0pB_taskb.txt,b,0,0,pagerank pr refers to both the concept and the...,train
7,g0pB_taskc.txt,c,3,1,vector space model is an algebraic model for r...,test
8,g0pB_taskd.txt,d,2,1,bayes theorem relates the conditional and marg...,train
9,g0pB_taske.txt,e,1,1,dynamic programming is a method for solving ma...,test



# Similarity Features 

One of the ways we might go about detecting plagiarism, is by computing **similarity features** that measure how similar a given answer text is as compared to the original wikipedia source text (for a specific task, a-e). The similarity features you will use are informed by [this paper on plagiarism detection](https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c412841_developing-a-corpus-of-plagiarised-short-answers/developing-a-corpus-of-plagiarised-short-answers.pdf). 
> In this paper, researchers created features called **containment** and **longest common subsequence**. 

Using these features as input, you will train a model to distinguish between plagiarized and not-plagiarized text files.

## Feature Engineering

Let's talk a bit more about the features we want to include in a plagiarism detection model and how to calculate such features. In the following explanations, I'll refer to a submitted text file as a **Student Answer Text (A)** and the original, wikipedia source file (that we want to compare that answer to) as the **Wikipedia Source Text (S)**.

### Containment

Your first task will be to create **containment features**. To understand containment, let's first revisit a definition of [n-grams](https://en.wikipedia.org/wiki/N-gram). An *n-gram* is a sequential word grouping. For example, in a line like "bayes rule gives us a way to combine prior knowledge with new information," a 1-gram is just one word, like "bayes." A 2-gram might be "bayes rule" and a 3-gram might be "combine prior knowledge."

> Containment is defined as the **intersection** of the n-gram word count of the Wikipedia Source Text (S) with the n-gram word count of the Student  Answer Text (S) *divided* by the n-gram word count of the Student Answer Text.

$$ \frac{\sum{count(\text{ngram}_{A}) \cap count(\text{ngram}_{S})}}{\sum{count(\text{ngram}_{A})}} $$

If the two texts have no n-grams in common, the containment will be 0, but if _all_ their n-grams intersect then the containment will be 1. Intuitively, you can see how having longer n-gram's in common, might be an indication of cut-and-paste plagiarism. In this project, it will be up to you to decide on the appropriate `n` or several `n`'s to use in your final model.

### Containment calculation

The general steps to complete this function are as follows:
1. From *all* of the text files in a given `df`, create an array of n-gram counts; it is suggested that you use a [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for this purpose.
2. Get the processed answer and source texts for the given `answer_filename`.
3. Calculate the containment between an answer and source text according to the following equation.

    >$$ \frac{\sum{count(\text{ngram}_{A}) \cap count(\text{ngram}_{S})}}{\sum{count(\text{ngram}_{A})}} $$
    
4. Return that containment value.

You are encouraged to write any helper functions that you need to complete the function below.

In [11]:
def calculate_containment(df, n, answer_filename):
    
    '''
    Calculates the containment between a given answer text and its associated source text.
    This function creates a count of ngrams (of a size, n) for each text file in our data.
    Then calculates the containment by finding the ngram count for a given answer text, 
    and its associated source text, and calculating the normalized intersection of those counts.
    
    :param df: 
        A dataframe with columns, 'File', 'Task', 'Category', 'Class', 'Text', and 'Datatype'
    :param n: 
        An integer that defines the ngram size
    :param answer_filename: 
        A filename for an answer text in the df, ex. 'g0pB_taskd.txt'
    :return: 
        A single containment value that represents the similarity between an answer text and its source text.
    '''
    
    df_answer = df.loc[complete_df['File'] == answer_filename]
    
    task = df_answer['Task'].values[0]
    a_text = df_answer['Text'].values[0]
    s_text = df.loc[(df['Task']==task) & (df['Datatype']=='orig')]['Text'].values[0]
    
    counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
    ngrams = counts.fit_transform([a_text, s_text]).toarray()
    
    containment = np.minimum(ngrams[0],ngrams[1]).sum() / ngrams[0].sum()

    return containment


In [12]:
n = 3
test_indices = range(5)

category_vals = []
containment_vals = []

for i in test_indices:
    category_vals.append(complete_df.loc[i, 'Category'])
    filename = complete_df.loc[i, 'File']
    c = calculate_containment(complete_df, n, filename)
    containment_vals.append(c)

print('Original category values: \n', category_vals)
print(str(n)+'-gram containment values: \n', containment_vals)

Original category values: 
 [0, 3, 2, 1, 0]

3-gram containment values: 
 [0.009345794392523364, 0.9641025641025641, 0.6136363636363636, 0.15675675675675677, 0.031746031746031744]


---
#### Because containment calculates the difference between the answer text and the original text in isolation (versus comparing answers to each other, or in combination to the source) it won't affect the output of the models when the data is split into training and testing sets.

---
## Longest Common Subsequence

Containment a good way to find overlap in word usage between two documents; it may help identify cases of cut-and-paste as well as paraphrased levels of plagiarism. Since plagiarism is a fairly complex task with varying levels, it's often useful to include other measures of similarity. The paper also discusses a feature called **longest common subsequence**.

> The longest common subsequence is the longest string of words (or letters) that are *the same* between the Wikipedia Source Text (S) and the Student Answer Text (A). This value is also normalized by dividing by the total number of words (or letters) in the  Student Answer Text. 

In this exercise, we'll ask you to calculate the longest common subsequence of words between two texts.

### EXERCISE: Calculate the longest common subsequence

* Given two texts: text A (answer text) of length n, and string S (original source text) of length m. Our goal is to produce their longest common subsequence of words: the longest sequence of words that appear left-to-right in both texts (though the words don't have to be in continuous order).
* Consider:
    * A = "i think pagerank is a link analysis algorithm used by google that uses a system of weights attached to each element of a hyperlinked set of documents"
    * S = "pagerank is a link analysis algorithm used by the google internet search engine that assigns a numerical weighting to each element of a hyperlinked set of documents"

* In this case, we can see that the start of each sentence of fairly similar, having overlap in the sequence of words, "pagerank is a link analysis algorithm used by" before diverging slightly. Then we **continue moving left -to-right along both texts** until we see the next common sequence; in this case it is only one word, "google". Next we find "that" and "a" and finally the same ending "to each element of a hyperlinked set of documents".

* Now, those words appear in left-to-right order in each document, sequentially, and even though there are some words in between, we count this as the longest common subsequence between the two texts. 
* If I count up each word that I found in common I get the value 20. **So, LCS has length 20**. 
* Next, to normalize this value, divide by the total length of the student answer; in this example that length is only 27. **So, the function `lcs_norm_word` should return the value `20/27` or about `0.7408`.**

In this way, LCS is a great indicator of cut-and-paste plagiarism or if someone has referenced the same source text multiple times in an answer.

### LCS, dynamic programming

If you read through the scenario above, you can see that this algorithm depends on looking at two texts and comparing them word by word. You can solve this problem in multiple ways. First, it may be useful to `.split()` each text into lists of comma separated words to compare. Then, you can iterate through each word in the texts and compare them, adding to your value for LCS as you go. 

The method I recommend for implementing an efficient LCS algorithm is: using a matrix and dynamic programming. **Dynamic programming** is all about breaking a larger problem into a smaller set of subproblems, and building up a complete result without having to repeat any subproblems. 

This approach assumes that you can split up a large LCS task into a combination of smaller LCS tasks.

### The matrix rules

One thing to notice here is that, you can efficiently fill up this matrix one cell at a time. Each grid cell only depends on the values in the grid cells that are directly on top and to the left of it, or on the diagonal/top-left. The rules are as follows:
* Start with a matrix that has one extra row and column of zeros.
* As you traverse your string:
    * If there is a match, fill that grid cell with the value to the top-left of that cell *plus* one. So, in our case, when we found a matching B-B, we added +1 to the value in the top-left of the matching cell, 0.
    * If there is not a match, take the *maximum* value from either directly to the left or the top cell, and carry that value over to the non-match cell.


After completely filling the matrix, **the bottom-right cell will hold the non-normalized LCS value**.

This matrix treatment can be applied to a set of words instead of letters. Your function should apply this to the words in two texts and return the normalized LCS value.

In [14]:
def lcs_norm_word(answer_text, source_text):
    '''
    Computes the longest common subsequence of words in two texts; returns a normalized value.
    
    :param answer_text: 
        The pre-processed text for an answer text
    :param source_text: 
        The pre-processed text for an answer's associated source text
    :return: 
        A normalized LCS value
    '''
    
    answer_text = answer_text.split()
    source_text = source_text.split()
    
    answer_len = len(answer_text)
    source_len = len(source_text)
    
    matrix = np.zeros((answer_len + 1, source_len + 1), dtype=int)
    
    for i in range(answer_len):
        for j in range(source_len):
            if answer_text[i] == source_text[j]:
                matrix[i+1][j+1] = matrix[i][j]+1
            else:
                matrix[i+1][j+1] = max(matrix[i+1][j], matrix[i][j+1])

    lcs = matrix[-1][-1] / answer_len
    
    return lcs 

In [17]:
test_indices = range(5)

category_vals = []
lcs_norm_vals = []

for i in test_indices:
    category_vals.append(complete_df.loc[i, 'Category'])
    answer_text = complete_df.loc[i, 'Text'] 
    task = complete_df.loc[i, 'Task']
    orig_rows = complete_df[(complete_df['Class'] == -1)]
    orig_row = orig_rows[(orig_rows['Task'] == task)]
    source_text = orig_row['Text'].values[0]

    lcs_val = lcs_norm_word(answer_text, source_text)
    lcs_norm_vals.append(lcs_val)

print('Original category values: \n', category_vals)
print('Normalized LCS values: \n', lcs_norm_vals)

Original category values: 
 [0, 3, 2, 1, 0]

Normalized LCS values: 
 [0.1917808219178082, 0.8207547169811321, 0.8464912280701754, 0.3160621761658031, 0.24257425742574257]


In [18]:
def create_containment_features(df, n, column_name=None):
    
    containment_values = []
    
    if(column_name==None):
        column_name = f'c_{n}'
    
    for i in df.index:
        file = df.loc[i, 'File']
        if df.loc[i,'Category'] > -1:
            c = calculate_containment(df, n, file)
            containment_values.append(c)
        else:
            containment_values.append(-1)
    
    print(f'{n}-gram containment features created!')
    return containment_values


In [19]:
def create_lcs_features(df, column_name='lcs_word'):
    
    lcs_values = []

    for i in df.index:
        if df.loc[i,'Category'] > -1:
            answer_text = df.loc[i, 'Text'] 
            task = df.loc[i, 'Task']
            orig_rows = df[(df['Class'] == -1)]
            orig_row = orig_rows[(orig_rows['Task'] == task)]
            source_text = orig_row['Text'].values[0]
            lcs = lcs_norm_word(answer_text, source_text)
            lcs_values.append(lcs)
        else:
            lcs_values.append(-1)

    print('LCS features created!')
    return lcs_values
    

In [20]:
ngram_range = range(1,7)
features_list = []

all_features = np.zeros((len(ngram_range)+1, len(complete_df)))

i=0
for n in ngram_range:
    column_name = 'c_'+str(n)
    features_list.append(column_name)
    all_features[i]=np.squeeze(create_containment_features(complete_df, n))
    i+=1

features_list.append('lcs_word')
all_features[i]= np.squeeze(create_lcs_features(complete_df))
features_df = pd.DataFrame(np.transpose(all_features), columns=features_list)

print('Features: ', features_list)

1-gram containment features created!
2-gram containment features created!
3-gram containment features created!
4-gram containment features created!
5-gram containment features created!
6-gram containment features created!
LCS features created!

Features:  ['c_1', 'c_2', 'c_3', 'c_4', 'c_5', 'c_6', 'lcs_word']



In [21]:
features_df.head(10)

,c_1,c_2,c_3,c_4,c_5,c_6,lcs_word
0,0.398148,0.079070,0.009346,0.000000,0.000000,0.000000,0.191781
1,1.000000,0.984694,0.964103,0.943299,0.922280,0.901042,0.820755
2,0.869369,0.719457,0.613636,0.515982,0.449541,0.382488,0.846491
3,0.593583,0.268817,0.156757,0.108696,0.081967,0.060440,0.316062
4,0.544503,0.115789,0.031746,0.005319,0.000000,0.000000,0.242574
5,0.329502,0.053846,0.007722,0.003876,0.000000,0.000000,0.161172
6,0.590308,0.150442,0.035556,0.004464,0.000000,0.000000,0.301653
7,0.765306,0.709898,0.664384,0.625430,0.589655,0.553633,0.621711
8,0.759777,0.505618,0.395480,0.306818,0.245714,0.195402,0.484305
9,0.884444,0.526786,0.340807,0.247748,0.180995,0.150000,0.597458


---
## Correlated Features

All of our features try to measure the similarity between two texts. Since our features are designed to measure similarity, it is expected that these features will be highly-correlated. Many classification models, for example a Naive Bayes classifier, rely on the assumption that features are *not* highly correlated; highly-correlated features may over-inflate the importance of a single feature. 

We want to choose your features based on which pairings have the lowest correlation. These correlation values range between 0 and 1; from low to high correlation, and are displayed in a [correlation matrix](https://www.displayr.com/what-is-a-correlation-matrix/), below.

In [22]:
corr_matrix = features_df.corr().abs().round(2)
display(corr_matrix)

,c_1,c_2,c_3,c_4,c_5,c_6,lcs_word
c_1,1.00,0.94,0.90,0.89,0.88,0.87,0.97
c_2,0.94,1.00,0.99,0.98,0.97,0.96,0.98
c_3,0.90,0.99,1.00,1.00,0.99,0.98,0.97
c_4,0.89,0.98,1.00,1.00,1.00,0.99,0.95
c_5,0.88,0.97,0.99,1.00,1.00,1.00,0.95
c_6,0.87,0.96,0.98,0.99,1.00,1.00,0.94
lcs_word,0.97,0.98,0.97,0.95,0.95,0.94,1.00


In [23]:
def train_test_data(complete_df, features_df, selected_features):
    '''
    Gets selected training and test features from given dataframes, and returns tuples for training and 
    test features and their corresponding class labels.
       
   :param complete_df: 
       A dataframe with all of our processed text data, datatypes, and labels
   :param features_df: 
       A dataframe of all computed, similarity features
   :param selected_features: 
       An array of selected features that correspond to certain columns in `features_df`
   :return: 
       training and test features and labels: (train_x, train_y), (test_x, test_y)
   '''
    features_df = features_df[selected_features]
    df = pd.concat([complete_df, features_df], axis=1) 
    
    train = df[df['Datatype'] == 'train']
    test = df[df['Datatype'] == 'test']

    train_x = train[selected_features].values
    train_y = train['Class'].values

    test_x = test[selected_features].values
    test_y = test['Class'].values
    
    return (train_x, train_y), (test_x, test_y)
    

In [26]:
selected_features = ['c_1', 'c_6', 'lcs_word']
(train_x, train_y), (test_x, test_y) = train_test_data(complete_df, features_df, selected_features)

print('Training size: ', len(train_x))
print('Test size: ', len(test_x))
print('Training df sample: \n', train_x[:10])

Training size:  70
Test size:  25

Training df sample: 
 [[0.39814815 0.         0.19178082]
 [0.86936937 0.38248848 0.84649123]
 [0.59358289 0.06043956 0.31606218]
 [0.54450262 0.         0.24257426]
 [0.32950192 0.         0.16117216]
 [0.59030837 0.         0.30165289]
 [0.75977654 0.1954023  0.48430493]
 [0.51612903 0.         0.27083333]
 [0.44086022 0.         0.22395833]
 [0.97945205 0.74468085 0.9       ]]


---
#### The selected features that I chose are being used because they have the least correlation among each other. Ideally, we want as much unique value from each feature to help identify true plagiarism, and higher correlation,  indicates that the results will identify similar patterns, leaving other patterns potentially unsurveilled. 

---
## Creating Final Data Files
We'll want to access the train and test data in SageMaker and upload it to S3. In this project, SageMaker will expect the following format for your train/test data:

* Training and test data should be saved in one `.csv` file each, ex `train.csv` and `test.csv`
* These files should have class  labels in the first column and features in the rest of the columns

This format follows the practice, outlined in the [SageMaker documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html), which reads: "Amazon SageMaker requires that a CSV file doesn't have a header record and that the target variable [class label] is in the first column."


In [37]:
def make_csv(x, y, filename, data_dir):
    '''
    Merges features and labels and converts them into one csv file with labels in the first column.
    
    :param x: 
        Data features
    :param y: 
        Data labels
    :param file_name: 
        Name of csv file, ex. 'train.csv'
    :param data_dir: 
        The directory where files will be saved
    '''
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    df = pd.concat([pd.DataFrame(y), pd.DataFrame(x)], axis=1)
    df = df.dropna()
    df.to_csv(os.path.join(data_dir, filename), header=None, index=False)
    
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [39]:
data_dir = 'plagiarism_data'

make_csv(train_x, train_y, filename='train.csv', data_dir=data_dir)
make_csv(test_x, test_y, filename='test.csv', data_dir=data_dir)

Path created: plagiarism_data/train.csv
Path created: plagiarism_data/test.csv
